In [1]:
from scipy import stats as st
import numpy as np
from itertools import islice
from math import log2
from collections import Counter
import matplotlib.pyplot as plt
from functools import reduce
from copy import deepcopy
from random import choice
from collections import namedtuple

In [ ]:
variant = 9
N = 200
# a = (-1) ** variant * 0.1 * variant
# sgm = (0.01 * variant + 1) ** 2
lmbd = 2 + (-1) ** variant * 0.01 * variant

In [ ]:
# normp = st.norm(a, sgm)
exp = st.expon(lmbd**(-1))

In [ ]:
st.expon.median(1/lmbd)

In [ ]:
# r = list(normp.rvs(size=N))
r = list(exp.rvs(size=N))
sr = sorted(r)

In [ ]:
m = int(1 + log2(N))
f, l = sr[0], sr[-1]
d = abs(l - f)
h = d / m
interv = [f + h * i for i in range(m)]
interv.append(l)
interv = list(zip(interv, islice(interv, 1, None)))

In [ ]:
interv

In [ ]:
first_counter = Counter()
for val in sr:
    first_counter[interv[findInterv(val, interv)]] += 1

In [ ]:
first_counter

In [ ]:
sec_counter = {(key[1] + key[0]) / 2: (val, val / N) for key, val in first_counter.items()}

In [ ]:
sec_counter

In [ ]:
w = [val[1] for key, val in sorted(sec_counter.items(), key=lambda x: x)]
sums = [round(reduce(lambda a, b: a + b, w[:i+1]), 5) for i in range(len(w))]
dsums = {el: i for i, el in enumerate(sums)}
try:
    prk = [(dsums[l], dsums[r]) for l, r in list(zip(sums, islice(sums, 1, None))) if l <= 0.5 < r][0]
except:
    prk = (0, 1)
a = [key for key, val in sorted(first_counter.items(), key=lambda x: (x[0]))]
med = a[prk[0]][1] if sums[prk[0]] == 0.5 else a[prk[0]][0] + (h / w[prk[0]]) * (0.5 - sums[prk[0]])

In [ ]:
med

In [ ]:
exp.median()

In [ ]:
list(zip(sums, islice(sums, 1, None)))

In [ ]:
w, sums, dsums

In [ ]:
hist(sec_counter, h, True)

In [2]:
def findInterv(val: float, arr: list) -> int:
    test = lambda x, item: True if item[0] <= x <= item[1] else False
    for i, item in enumerate(arr):
        if test(val, item):
            return i
    return None

In [3]:
def hist(sec_counter: dict, h: float, show: bool = False, path: str = 'Data/hist.png'):
    plt.bar(list(sec_counter.keys()), [val[1] / h for val in sec_counter.values()], color='b', edgecolor='black')
    if show:
        plt.show()
    else:
        plt.savefig(path)

In [4]:
def cdf(sr: list, N: int, sizex: int = 30, sizey: int = 30, show: bool = False, path: str = 'Data/cdf.png'):

    items = list(zip(sr, islice(sr, 1, None)))
    delta = 0.00001
    Xlist = [[x * delta for x in range(int(item[0] / delta), int(item[1] / delta))] 
             for item in items]
    Ylist = [[i / N] * len(Xlist[i]) for i in range(N-1)]

    ##############################################################

    fig, ax = plt.subplots(figsize=(sizex, sizey))

    for X, Y in zip(Xlist, Ylist):
        ax.plot(X, Y, label='', color='black')
    ax.set_title('Эмпирическая функция распределения')
    ax.legend(loc='upper left')
    ax.set_ylabel('y')
    ax.set_xlabel('x')
    ax.set_xlim(xmin=sr[0], xmax=sr[-1])
    ax.set_ylim(ymin=0, ymax=1)
    fig.tight_layout()
    if sizex != 5:
        ax.set_axisbelow(True)
        ax.minorticks_on()
        ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
        ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
    else:
        ax.grid()
    if show:
        plt.show()
    else:
        plt.savefig(path)

In [5]:
def expStats(sec_counter, first_counter, h):
    tmp = list(sec_counter.items())
    tmp.insert(0, 0.0)
    mean = reduce(lambda a, b: a + b[0] * b[1][1], tmp)
    s2 = reduce(lambda a, b: a + (b[0] - mean) ** 2 * b[1][1], tmp) - (h ** 2) / 12
    ds = s2 ** 0.5
    
    ak, most_common = first_counter.most_common(1)[0]
    wn = sorted(first_counter.items(), key=lambda x: x[0])
    k = {val: i for i, (key, val) in enumerate(sorted(first_counter.items(), key=lambda x: x[0]))}[most_common]
    wk = lambda k: sorted(sec_counter.items(), key=lambda x: x[0])[k][1][1]
    mode = ak[0] + h * ((wk(k) - wk(k-1)) / (2 * wk(k) - wk(k-1) - wk(k+1)))
    
    w = [val[1] for key, val in sorted(sec_counter.items(), key=lambda x: x)]
    sums = [reduce(lambda a, b: a + b, w[:i+1]) for i in range(len(w))]
    dsums = {el: i for i, el in enumerate(sums)}
    try:
        prk = [(dsums[l], dsums[r]) for l, r in list(zip(sums, islice(sums, 1, None))) if l <= 0.5 < r][0]
        a = [key for key, val in sorted(first_counter.items(), key=lambda x: (x[0]))]
        med = a[prk[0]][1] if sums[prk[0]] == 0.5 else a[prk[0]][0] + (h / w[prk[0]]) * (0.5 - sums[prk[0]])
    except:
        prk = (0, 1)
        a = [key for key, val in sorted(first_counter.items(), key=lambda x: (x[0]))]
        med = a[prk[0]][1] if sums[prk[0]] == 0.5 else a[prk[0]][0] + (h / w[prk[0]]) * (0.5 - sums[prk[0]])
        med = abs(med*10)
    
    mk = lambda k: sum(map(lambda item: item[0] ** k * item[1][1], sec_counter.items()))
    mck = lambda k: sum(map(lambda item: (item[0] - mean) ** k * item[1][1], sec_counter.items()))
    skew, kurtosis = mck(3) / (ds ** 3), (mck(4) / (ds ** 4)) - 3
    
    return mean, s2, ds, mode, med, skew, kurtosis

In [6]:
class LabFitter3000(object):
    '''IN: distribution name, params, scipy class
    OUT: path to figs, experimental stats, theoretical stats'''
    def __init__(self, obj, N, name, *params):
        self.distribution = obj
        self.N = N
        self.distribution_name = name
        self.params = params
        self.rvs = None
        self.h = None
        self.first_counter = None
        self.second_counter = None
        self.pathes = []
        self.experimental_stats = None
        self.theoretical_stats = None
        self.Stats = namedtuple('Stats', 'mean variance std mode med skew kurtosis')

    def create_rvs(self):
        self.rvs = list(self.distribution.rvs(size=self.N))

    def create_first_counter(self):
        m = int(1 + log2(self.N))
        sr = sorted(self.rvs)
        f, l = sr[0], sr[-1]
        d = abs(l - f)
        self.h = d / m
        interv = [f + self.h * i for i in range(m)]
        interv.append(l)
        interv = list(zip(interv, islice(interv, 1, None)))

        self.first_counter = Counter()
        for val in sr:
            self.first_counter[interv[findInterv(val, interv)]] += 1

    def create_second_counter(self):
        self.sec_counter = {(key[1] + key[0]) / 2: (val, val / self.N)
                            for key, val in self.first_counter.items()}

    def hist(self, show: bool = False):#, sec_counter: dict, h: float, show: bool = False, path: str = 'Data/hist.png'):
        tmp = {key: val[1] / self.h for key, val in self.sec_counter.items()}
#         plt.bar(list(self.sec_counter.keys()), [val[1] / self.h for val in self.sec_counter.values()], color='b', edgecolor='black')
        plt.bar(list(tmp.keys()),
                list(tmp.values()),
                color='b',
                edgecolor='black',
                width=self.h)
        if show:
            plt.show()
        else:
            path = 'Data/' + self.distribution_name + '_hist.png'
            plt.savefig(path)
            plt.clf()

    def cdf(self, show: bool = False, sizex: int = 30, sizey: int = 30):#, sr: list, N: int, sizex: int = 30, sizey: int = 30, show: bool = False, path: str = 'Data/cdf.png'):

        sr = sorted(self.rvs)
        items = list(zip(sr, islice(sr, 1, None)))
        delta = 0.00001
        Xlist = [[x * delta for x in range(int(item[0] / delta), int(item[1] / delta))]
                 for item in items]
        Ylist = [[i / self.N] * len(Xlist[i]) for i in range(self.N-1)]

        ##############################################################

        fig, ax = plt.subplots(figsize=(sizex, sizey))

        for X, Y in zip(Xlist, Ylist):
            ax.plot(X, Y, label='', color='black')
        ax.set_title('Эмпирическая функция распределения')
        ax.legend(loc='upper left')
        ax.set_ylabel('y')
        ax.set_xlabel('x')
        ax.set_xlim(xmin=sr[0], xmax=sr[-1])
        ax.set_ylim(ymin=0, ymax=1)
        fig.tight_layout()
        if sizex != 5:
            ax.set_axisbelow(True)
            ax.minorticks_on()
            ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
            ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
        else:
            ax.grid()
        if show:
            plt.show()
        else:
            path = 'Data/' + self.distribution_name + '_cdf.png'
            plt.savefig(path)
            plt.clf()

    def create_theoretical_stats(self, mode_generator):
        mean, variance, skew, kurtosis = self.distribution.stats(moments='mvsk')
        med = self.distribution.median()
        std = self.distribution.std()
        mode = mode_generator(*self.params)

        self.theoretical_stats = self.Stats(mean=float(mean),
                                            variance=float(variance),
                                            std=std,
                                            mode=mode,
                                            med=med,
                                            skew=float(skew),
                                            kurtosis=float(kurtosis))

    def create_experimental_stats(self, exp_stats_generator):
        mean, variance, std, mode, med, skew, kurtosis = exp_stats_generator(self.sec_counter,
                                                                             self.first_counter,
                                                                             self.h)
        self.experimental_stats = self.Stats(mean=mean,
                                            variance=variance,
                                            std=std,
                                            mode=mode,
                                            med=med,
                                            skew=skew,
                                            kurtosis=kurtosis)

In [7]:
def fitter(obj, mode_generator):
    obj.create_rvs()
    obj.create_first_counter()
    obj.create_second_counter()
#     obj.draw_hist(hist)
#     obj.draw_cdf(cdf) 
    obj.hist()
    obj.cdf()
    obj.create_theoretical_stats(mode_generator)
    obj.create_experimental_stats(expStats)
    return obj

In [129]:
variant = 9
N = 200

mode = {'norm': lambda a, sgm: a,
        'expon': lambda lmbd: 0,
        'uniform': lambda a, b: sum([a, b])/2}

Obj = namedtuple('Obj', 'name params distribution')

mu = (-1) ** variant * 0.1 * variant
sgm = (0.01 * variant + 1) ** 2

lmbd = 2 + (-1) ** variant * 0.01 * variant

a = (-1) ** variant * 0.05 * variant
b = a + 0.05 * variant + 1

Data = [Obj(name='norm',
            params=[mu, sgm],
            distribution=st.norm(mu, sgm)),
        Obj(name='expon',
            params=[1/lmbd],
            distribution=st.expon(1/lmbd)),
        Obj(name='uniform',
            params=[a, b],
            distribution=st.uniform(a, b))]

result = []
for obj in Data:
    result.append(LabFitter3000(obj.distribution, N, obj.name, *obj.params))
    
result = [fitter(lf, mode[Data[i].name]) for i, lf in enumerate(result)]   

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/matplotlib/axes/_axes.py:545: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


In [10]:
for el in result:
    print(el.experimental_stats, el.theoretical_stats)

Stats(mean=-0.9359117594525188, variance=1.3682179217489618, std=1.1697084772493367, mode=-0.6095155336528975, med=-1.5420422810575807, skew=-0.41301283788640636, kurtosis=0.14597702411247493) Stats(mean=-0.9, variance=1.4115816100000003, std=1.1881000000000002, mode=-0.9, med=-0.9, skew=0.0, kurtosis=0.0)
Stats(mean=1.6319500370938527, variance=0.9539084905919628, std=0.9766823898238172, mode=1.061016324098197, med=4.875453917121658, skew=2.2313318208644417, kurtosis=6.494917072702231) Stats(mean=1.5235602094240837, variance=1.0, std=1.0, mode=0, med=1.216707389984029, skew=2.0, kurtosis=6.0)
Stats(mean=0.0018940583698077527, variance=0.07799162694936031, std=0.27926981030781023, mode=-0.15504578261492588, med=-0.18014393546327012, skew=0.2509985384584842, kurtosis=-1.1305139794604604) Stats(mean=0.04999999999999999, variance=0.08333333333333333, std=0.28867513459481287, mode=0.275, med=0.04999999999999999, skew=0.0, kurtosis=-1.2)


In [ ]:
normd = LabFitter3000(st.norm(a, sgm), N, 'norm', a, sgm)
fitter(normd, mode['norm'])

In [ ]:
uniformd = LabFitter3000(st.uniform(a, b), N, 'uniform', a, b)
fitter(uniformd, mode['uniform'])

In [ ]:
expond = LabFitter3000(st.expon(loc=0, scale=1/lmbd), N, 'expon', 1/lmbd)
fitter(expond, mode['expon'])

In [ ]:
uniformd.experimental_stats

In [ ]:
uniformd.theoretical_stats

In [ ]:
lmbd

In [ ]:
from math import log

In [ ]:
log(2) / (lmbd)

In [ ]:
a, b

In [11]:
a = np.asarray([ [1,2,3], [4,5,6], [7,8,9] ])
np.savetxt("foo.csv", a, delimiter=",")

In [12]:
a

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [13]:
a = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [18]:
a = a.reshape(3, -1)

In [34]:
np.savetxt("foo.csv", a, delimiter=";")

In [31]:
a = np.array(result[1].rvs)

In [32]:
a = a.reshape(20, -1)

In [33]:
a.shape

(20, 10)

In [36]:
my_dict = {"test": [1, 2], "testing": [3, 4]}

my_dict

{'test': [1, 2], 'testing': [3, 4]}

In [45]:
a = np.asarray(list(zip(my_dict.keys(), my_dict.values())))

ValueError: setting an array element with a sequence

In [46]:
list(zip(my_dict.keys(), my_dict.values()))

[('test', [1, 2]), ('testing', [3, 4])]

In [44]:
a

array(<zip object at 0x113d8e188>, dtype=object)

In [48]:
import pandas as pd
df = pd.DataFrame(my_dict)

In [50]:
df.to_csv('trtrtr.csv', sep=';', encoding='utf-8')

In [63]:
rr = result[0].experimental_stats._asdict()

In [64]:
rr

OrderedDict([('mean', -0.9359117594525188),
             ('variance', 1.3682179217489618),
             ('std', 1.1697084772493367),
             ('mode', -0.6095155336528975),
             ('med', -1.5420422810575807),
             ('skew', -0.41301283788640636),
             ('kurtosis', 0.14597702411247493)])

In [65]:
r = dict(rr)

In [66]:
r

{'kurtosis': 0.14597702411247493,
 'mean': -0.9359117594525188,
 'med': -1.5420422810575807,
 'mode': -0.6095155336528975,
 'skew': -0.41301283788640636,
 'std': 1.1697084772493367,
 'variance': 1.3682179217489618}

In [88]:
rr = result[0].experimental_stats._asdict()
df = pd.DataFrame(list(rr.values()), index=rr.keys(), columns=[['values']])

In [89]:
df

,values
mean,-0.935912
variance,1.368218
std,1.169708
mode,-0.609516
med,-1.542042
skew,-0.413013
kurtosis,0.145977


In [79]:
list(rr.values())

[-0.9359117594525188,
 1.3682179217489618,
 1.1697084772493367,
 -0.6095155336528975,
 -1.5420422810575807,
 -0.41301283788640636,
 0.14597702411247493]

In [100]:
allst = list(zip(result[0].experimental_stats._asdict().keys(), list(zip(result[0].experimental_stats, result[0].theoretical_stats))))

In [103]:
result = {key: (e, t, abs(e-t), abs(e-t)/t) for key, (e, t) in allst}
result

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in double_scalars
  """Entry point for launching an IPython kernel.


{'kurtosis': (0.14597702411247493, 0.0, 0.14597702411247493, inf),
 'mean': (-0.9359117594525188, -0.9, 0.03591175945251879, -0.0399019549472431),
 'med': (-1.5420422810575807, -0.9, 0.6420422810575807, -0.7133803122862007),
 'mode': (-0.6095155336528975, -0.9, 0.29048446634710257, -0.3227605181634473),
 'skew': (-0.41301283788640636, 0.0, 0.41301283788640636, inf),
 'std': (1.1697084772493367,
  1.1881000000000002,
  0.01839152275066347,
  0.015479776744940213),
 'variance': (1.3682179217489618,
  1.4115816100000003,
  0.043363688251038424,
  0.030719930001807276)}

In [106]:
pd.DataFrame(list(result.values()), index=result.keys(), columns=[['experimental'], ['theoretical'], ['absolute'], ['relative']])

,experimental
,theoretical
,absolute
,relative
kurtosis,0.145977
skew,-0.413013
mean,-0.935912
mode,-0.609516
variance,1.368218
med,-1.542042
std,1.169708


In [109]:
pd.DataFrame(result)

,kurtosis,mean,med,mode,skew,std,variance
0,0.145977,-0.935912,-1.542042,-0.609516,-0.413013,1.169708,1.368218
1,0.000000,-0.900000,-0.900000,-0.900000,0.000000,1.188100,1.411582
2,0.145977,0.035912,0.642042,0.290484,0.413013,0.018392,0.043364
3,inf,-0.039902,-0.713380,-0.322761,inf,0.015480,0.030720


In [110]:
result

{'kurtosis': (0.14597702411247493, 0.0, 0.14597702411247493, inf),
 'mean': (-0.9359117594525188, -0.9, 0.03591175945251879, -0.0399019549472431),
 'med': (-1.5420422810575807, -0.9, 0.6420422810575807, -0.7133803122862007),
 'mode': (-0.6095155336528975, -0.9, 0.29048446634710257, -0.3227605181634473),
 'skew': (-0.41301283788640636, 0.0, 0.41301283788640636, inf),
 'std': (1.1697084772493367,
  1.1881000000000002,
  0.01839152275066347,
  0.015479776744940213),
 'variance': (1.3682179217489618,
  1.4115816100000003,
  0.043363688251038424,
  0.030719930001807276)}

In [95]:
list(zip(result[0].experimental_stats._asdict().keys(), result[0].experimental_stats))

[('mean', -0.9359117594525188),
 ('variance', 1.3682179217489618),
 ('std', 1.1697084772493367),
 ('mode', -0.6095155336528975),
 ('med', -1.5420422810575807),
 ('skew', -0.41301283788640636),
 ('kurtosis', 0.14597702411247493)]

In [97]:
result[0].experimental_stats

Stats(mean=-0.9359117594525188, variance=1.3682179217489618, std=1.1697084772493367, mode=-0.6095155336528975, med=-1.5420422810575807, skew=-0.41301283788640636, kurtosis=0.14597702411247493)

In [120]:
df = pd.DataFrame(result)


In [121]:
df

,kurtosis,mean,med,mode,skew,std,variance
0,0.145977,-0.935912,-1.542042,-0.609516,-0.413013,1.169708,1.368218
1,0.000000,-0.900000,-0.900000,-0.900000,0.000000,1.188100,1.411582
2,0.145977,0.035912,0.642042,0.290484,0.413013,0.018392,0.043364
3,inf,-0.039902,-0.713380,-0.322761,inf,0.015480,0.030720


In [123]:
df.pivot_table(index=['experimental', 'theoretical', 'absolute', 'relative']).reset_index()

,index,kurtosis,mean,med,mode,skew,std,variance
0,absolute,0.145977,0.035912,0.642042,0.290484,0.413013,0.018392,0.043364
1,experimental,0.145977,-0.935912,-1.542042,-0.609516,-0.413013,1.169708,1.368218
2,relative,inf,-0.039902,-0.713380,-0.322761,inf,0.015480,0.030720
3,theoretical,0.000000,-0.900000,-0.900000,-0.900000,0.000000,1.188100,1.411582


In [128]:
pd.DataFrame(list(result.values()), index=list(result.keys()))

,0,1,2,3
kurtosis,0.145977,0.000000,0.145977,inf
skew,-0.413013,0.000000,0.413013,inf
mean,-0.935912,-0.900000,0.035912,-0.039902
mode,-0.609516,-0.900000,0.290484,-0.322761
variance,1.368218,1.411582,0.043364,0.030720
med,-1.542042,-0.900000,0.642042,-0.713380
std,1.169708,1.188100,0.018392,0.015480


In [141]:
pd.DataFrame(result[0].first_counter)

ValueError: If using all scalar values, you must pass an index

In [138]:
dict(result[0].first_counter)

{(-3.4434993924798873, -2.7425233603043733): 9,
 (-2.7425233603043733, -2.0415473281288588): 23,
 (-2.0415473281288588, -1.3405712959533447): 41,
 (-1.3405712959533447, -0.6395952637778302): 50,
 (-0.6395952637778302, 0.06138076839768436): 35,
 (0.06138076839768436, 0.762356800573198): 20,
 (0.762356800573198, 1.4633328327487125): 17,
 (1.4633328327487125, 2.1643088649242275): 5}

In [150]:
res = {str((round(key[0], 4), round(key[1], 4))): [val] for key, val in result[0].first_counter.items()}
pd.DataFrame(res)

,"(-0.6396, 0.0614)","(-1.3406, -0.6396)","(-2.0415, -1.3406)","(-2.7425, -2.0415)","(-3.4435, -2.7425)","(0.0614, 0.7624)","(0.7624, 1.4633)","(1.4633, 2.1643)"
0,35,50,41,23,9,20,17,5


In [154]:
pd.DataFrame(result[0].sec_counter)

,-3.0930113763921305,-2.392035344216616,-1.6910593120411017,-0.9900832798655874,-0.2891072476900729,0.4118687844854412,1.1128448166609552,1.81382084883647
0,9.000,23.000,41.000,50.00,35.000,20.0,17.000,5.000
1,0.045,0.115,0.205,0.25,0.175,0.1,0.085,0.025


In [153]:
result[0].sec_counter

{-3.0930113763921305: (9, 0.045),
 -2.392035344216616: (23, 0.115),
 -1.6910593120411017: (41, 0.205),
 -0.9900832798655874: (50, 0.25),
 -0.2891072476900729: (35, 0.175),
 0.4118687844854412: (20, 0.1),
 1.1128448166609552: (17, 0.085),
 1.81382084883647: (5, 0.025)}

In [145]:
dict(result[0].first_counter)

{(-3.4434993924798873, -2.7425233603043733): 9,
 (-2.7425233603043733, -2.0415473281288588): 23,
 (-2.0415473281288588, -1.3405712959533447): 41,
 (-1.3405712959533447, -0.6395952637778302): 50,
 (-0.6395952637778302, 0.06138076839768436): 35,
 (0.06138076839768436, 0.762356800573198): 20,
 (0.762356800573198, 1.4633328327487125): 17,
 (1.4633328327487125, 2.1643088649242275): 5}

In [146]:
data

{1: [1, 2, 3], 2: [1, 2, 3]}

In [161]:
shape = 20
rvs = np.array(result[0].rvs).reshape(shape, -1)
rvs.tofile('rvs.csv',
           sep=';',
           format='%10.5f')


In [ ]:
import docx 
import csv

doc = docx.Document()

with open('csv.csv', newline='') as f:
    csv_reader = csv.reader(f) 

    csv_headers = next(csv_reader)
    csv_cols = len(csv_headers)

    table = doc.add_table(rows=2, cols=csv_cols)
    hdr_cells = table.rows[0].cells

    for i in range(csv_cols):
        hdr_cells[i].text = csv_headers[i]

    for row in csv_reader:
        row_cells = table.add_row().cells
        for i in range(csv_cols):
            row_cells[i].text = row[i]

doc.add_page_break()
doc.save("data.docx")

In [1]:
import numpy as np

In [5]:
rvs = np.array([i for i in range(200)])
print(rvs, '\n')
rvs = rvs.reshape(20, -1)
print(rvs)
rvs.tofile('trtrtr.csv',
            sep=';',
            format='%10.5f')

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199] 

[[  0   1   2   3   4   5   6   7   8   9]
 [ 10  11  12  13  14  15  16  17  18  19]
 [ 20  21  22  23  24  25  26  27  28  29]
 [ 30  31  32  33  34  35  36  37  38  39]
 [ 40  41  42

In [6]:
import pandas as pd

In [17]:
df = pd.DataFrame(rvs, columns=['a b c d e f g h i j'.split()])
df.to_csv('trtrtr.csv', sep=';', encoding='utf-8', index=False)

TypeError: to_csv() got an unexpected keyword argument 'format'

In [15]:
import string
list(string.ascii_lowercase)[:10]

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']

In [16]:
import pandas as pd

In [49]:
norm_r = st.norm(-0.9, 1.1881)
expon_r = st.expon(1/1.91)
uniform_r = st.uniform(-0.45, 1.0)

In [50]:
norm_w = [0.005, 0.035, 0.13, 0.205, 0.29, 0.165, 0.115, 0.055]
expon_w = [0.58, 0.27, 0.105, 0.005, 0.025, 0.005, 0.005, 0.005]
uniform_w = [0.15, 0.14, 0.105, 0.12, 0.155, 0.095, 0.115, 0.12]

In [51]:
norm_x = [(-0.4032, 0.394), (-1.2004, -0.4032), (-1.9976, -1.2004), (-2.7947, -1.9976), (-3.5919, -2.7947), (-4.3891, -3.5919), (0.394, 1.1911), (1.1911, 1.9883)]
expon_x = [(0.5261, 1.4781), (1.4781, 2.4301), (2.4301, 3.3821), (3.3821, 4.3341), (4.3341, 5.2861), (5.2861, 6.2381), (6.2381, 7.1901), (7.1901, 8.1421)]
uniform_x = [(-0.0744, 0.0504), (-0.1993, -0.0744), (-0.3242, -0.1993), (-0.449, -0.3242), (0.0504, 0.1753), (0.1753, 0.3002), (0.3002, 0.425), (0.425, 0.5499)]

In [52]:
r = (norm_r, expon_r, uniform_r)
w = (norm_w, expon_w, uniform_w)
x = (norm_x, expon_x, uniform_x)

In [60]:
res = []
for cur_r, cur_w, cur_x in [(*ab, c) for ab, c in list(zip(zip(r, w), x))]:
    print(cur_r, cur_w, cur_x)
    p = [cur_r.cdf(b) - cur_r.cdf(a) for a, b in cur_x]
    s = sum(p)
    wp = [abs(a - b) for a, b in zip(cur_w, p)]
    m = max(wp)
    new_w = deepcopy(cur_w)
    new_w.append(1.0)
    p.append(s)
    wp.append(m)
    new_x = deepcopy(cur_x)
    new_x.append('-')
    res.append(pd.DataFrame(np.round(np.array([new_w, p, wp]).T, 4), index=new_x))

<scipy.stats._distn_infrastructure.rv_frozen object at 0x116099160> [0.005, 0.035, 0.13, 0.205, 0.29, 0.165, 0.115, 0.055] [(-0.4032, 0.394), (-1.2004, -0.4032), (-1.9976, -1.2004), (-2.7947, -1.9976), (-3.5919, -2.7947), (-4.3891, -3.5919), (0.394, 1.1911), (1.1911, 1.9883)]
<scipy.stats._distn_infrastructure.rv_frozen object at 0x115f75cc0> [0.58, 0.27, 0.105, 0.005, 0.025, 0.005, 0.005, 0.005] [(0.5261, 1.4781), (1.4781, 2.4301), (2.4301, 3.3821), (3.3821, 4.3341), (4.3341, 5.2861), (5.2861, 6.2381), (6.2381, 7.1901), (7.1901, 8.1421)]
<scipy.stats._distn_infrastructure.rv_frozen object at 0x1160996d8> [0.15, 0.14, 0.105, 0.12, 0.155, 0.095, 0.115, 0.12] [(-0.0744, 0.0504), (-0.1993, -0.0744), (-0.3242, -0.1993), (-0.449, -0.3242), (0.0504, 0.1753), (0.1753, 0.3002), (0.3002, 0.425), (0.425, 0.5499)]


In [61]:
res[0]

,0,1,2
"(-0.4032, 0.394)",0.005,0.1999,0.1949
"(-1.2004, -0.4032)",0.035,0.2619,0.2269
"(-1.9976, -1.2004)",0.130,0.2224,0.0924
"(-2.7947, -1.9976)",0.205,0.1224,0.0826
"(-3.5919, -2.7947)",0.290,0.0437,0.2463
"(-4.3891, -3.5919)",0.165,0.0101,0.1549
"(0.394, 1.1911)",0.115,0.0988,0.0162
"(1.1911, 1.9883)",0.055,0.0317,0.0233
-,1.000,0.9908,0.2463


In [62]:
res[1]

,0,1,2
"(0.5261, 1.4781)",0.580,0.6125,0.0325
"(1.4781, 2.4301)",0.270,0.2364,0.0336
"(2.4301, 3.3821)",0.105,0.0912,0.0138
"(3.3821, 4.3341)",0.005,0.0352,0.0302
"(4.3341, 5.2861)",0.025,0.0136,0.0114
"(5.2861, 6.2381)",0.005,0.0052,0.0002
"(6.2381, 7.1901)",0.005,0.0020,0.0030
"(7.1901, 8.1421)",0.005,0.0008,0.0042
-,1.000,0.9970,0.0336


In [63]:
res[2]

,0,1,2
"(-0.0744, 0.0504)",0.150,0.1248,0.0252
"(-0.1993, -0.0744)",0.140,0.1249,0.0151
"(-0.3242, -0.1993)",0.105,0.1249,0.0199
"(-0.449, -0.3242)",0.120,0.1248,0.0048
"(0.0504, 0.1753)",0.155,0.1249,0.0301
"(0.1753, 0.3002)",0.095,0.1249,0.0299
"(0.3002, 0.425)",0.115,0.1248,0.0098
"(0.425, 0.5499)",0.120,0.1249,0.0049
-,1.000,0.9989,0.0301


In [64]:
for df, type_r in zip(res, ['norm', 'expon', 'uniform']):
    df.to_csv(type_r + '.csv', sep=';', encoding='utf-8')